# 11 ODE integrators: Verlet (students)

In [ ]:
from importlib import reload

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')

import integrators

In [ ]:
reload(integrators)

## Velocity Verlet

Use expansion *forward* and *backward* (!) in time (Hamiltons (i.e. Newton without friction) equations are time symmetric)

\begin{align}
r(t + \Delta r) &\approx r(t) + \Delta t\, v(t) + \frac{1}{2m} \Delta t^2 F(t)\\
r(t) &\approx r(t + \Delta t) - \Delta t\, v(t + \Delta t) + \frac{1}{2m} \Delta t^2 F(t+\Delta t)
\end{align}

Solve for $v$:
\begin{align}
v(t+\Delta t) &\approx v(t) + \frac{1}{2m} \Delta t \big(F(t) + F(t+\Delta t)\big)
\end{align}

Complete **Velocity Verlet** integrator consists of the first and third equation.

In practice, split into three steps (calculate the velocity at the half time step):
\begin{align}
v(t+\frac{\Delta t}{2}) &= v(t) + \frac{\Delta t}{2} \frac{F(t)}{m} \\
r(t + \Delta r) &= r(t) + \Delta t\, v(t+\frac{\Delta t}{2})\\
v(t+\Delta t) &= v(t+\frac{\Delta t}{2}) + \frac{\Delta t}{2} \frac{F(t+\Delta t)}{m}
\end{align}

When writing production-level code, remember to re-use $F(t+\Delta t)$ als the "new" starting $F(t)$ in the next iteration (and don't recompute).

### Integration of planetary motion 
Gravitational potential energy:
$$
U(r) = -\frac{GMm}{r}
$$
with $r$ the distance between the two masses $m$ and $M$.

#### Central forces
$$
U(\mathbf{r}) = f(r) = f(\sqrt{\mathbf{r}\cdot\mathbf{r}})\\
\mathbf{F} = -\nabla U(\mathbf{r}) = -\frac{\partial f(r)}{\partial r} \, \frac{\mathbf{r}}{r} 
$$

#### Force of gravity
\begin{align}
\mathbf{F} &= -\frac{G m M}{r^2} \hat{\mathbf{r}}\\
\hat{\mathbf{r}} &= \frac{1}{\sqrt{x^2 + y^2}} \left(\begin{array}{c} x \\ y \end{array}\right)
\end{align}

#### Integrate simple planetary orbits 
Set $$GM = 1$$ and try initial conditions
$$
x(0) = 0.5, \quad y(0)=0, \quad v_x(0)=0, \quad v_y(0)=1.63
$$

In [ ]:
def F_gravity(r, m=1, G=1, M=1):
    #

def U_gravity(r, m=1, G=1, M=1):
    return -G*m*M/np.sqrt(np.sum(r*r))

In [ ]:
# 2D planetary motion with velocity verlet


In [ ]:
# plot orbit

## Velocity Verlet vs RK4: Energy conservation

### Implement gravity force in `integrators2.py`
Add `F_gravity` to the `integrators2.py` module. Use the new function `unitvector()`.

### Planetary orbits with `integrators2.py` 

In [ ]:
r0 = np.array([0.5, 0])
v0 = np.array([0, 1.63])

In [ ]:
import integrators2
from importlib import reload
reload(integrators2)

Use the new function `integrators2.integrate_newton_2d()` to integrate 2d coordinates.

#### RK4

In [ ]:
trk4, yrk4 = integrators2.integrate_newton_2d(x0=r0, v0=v0, t_max=100, mass=1,
                                       h=0.01,
                                       force=integrators2.F_gravity, 
                                       integrator=integrators2.rk4)

In [ ]:
rxrk4, ryrk4 = yrk4[:, 0, 0], yrk4[:, 0, 1]
ax = plt.subplot(1,1,1)
ax.set_aspect(1)
ax.plot(rxrk4, ryrk4)

In [ ]:
integrators2.analyze_energies(trk4, yrk4, integrators2.U_gravity)

In [ ]:
print("Energy conservation RK4 for {} steps: {}".format(
        len(trk4),
        integrators2.energy_conservation(trk4, yrk4, integrators2.U_gravity)))

#### Euler 

In [ ]:
te, ye = integrators2.integrate_newton_2d(x0=r0, v0=v0, t_max=100, mass=1,
                                         h=0.01,
                            force=F_gravity, 
                            integrator=integrators2.euler)
rex, rey = ye[:, 0].T

In [ ]:
ax = plt.subplot(1,1,1)
ax.plot(rx, ry, label="RK4")
ax.plot(rex, rey, label="Euler")
ax.legend(loc="best")

In [ ]:
integrators2.analyze_energies(te, ye, integrators2.U_gravity)

In [ ]:
print("Energy conservation Euler for {} steps: {}".format(
        len(te),
        integrators2.energy_conservation(te, ye, integrators2.U_gravity)))

#### Velocity Verlet

In [ ]:
tv, yv = integrators2.integrate_newton_2d(x0=r0, v0=v0, t_max=100, mass=1,
                                       h=0.01,
                                       force=F_gravity, 
                                       integrator=integrators2.velocity_verlet)

In [ ]:
rxv, ryv = yv[:, 0].T
ax = plt.subplot(1,1,1)
ax.set_aspect(1)
ax.plot(rxv, ryv, label="velocity Verlet")
ax.plot(rxrk4, ryrk4, label="RK4")
ax.legend(loc="best")

In [ ]:
integrators2.analyze_energies(tv, yv, integrators2.U_gravity)

In [ ]:
print("Energy conservation Velocity Verlet for {} steps: {}".format(
        len(tv),
        integrators2.energy_conservation(tv, yv, integrators2.U_gravity)))

#### Longer time scale stability
Run RK4 and Velocity Verlet for longer.

**NOTE: Runs *much* longer**

In [ ]:
tv2, yv2 = integrators2.integrate_newton_2d(x0=r0, v0=v0, t_max=10000, mass=1,
                                       h=0.01,
                                       force=F_gravity, 
                                       integrator=integrators2.velocity_verlet)

In [ ]:
print("Energy conservation Velocity Verlet for {} steps: {}".format(
        len(tv2),
        integrators2.energy_conservation(tv2, yv2, integrators2.U_gravity)))

In [ ]:
t4, y4 = integrators2.integrate_newton_2d(x0=r0, v0=v0, t_max=10000, mass=1,
                                       h=0.01,
                                       force=F_gravity, 
                                       integrator=integrators2.rk4)

In [ ]:
print("Energy conservation RK4 for {} steps: {}".format(
        len(t4),
        integrators2.energy_conservation(t4, y4, integrators2.U_gravity)))